In [1]:
import requests
import re
import os
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import pandas as pd
import datetime

## 爬PTT標題

In [2]:
def get_board_url(Board_Name):
    url = urllib.parse.urljoin('https://www.ptt.cc/bbs/', Board_Name, '/index.html')
    return url

In [3]:
def page_num(url, num):
    article = get_articles(url, today)
    bb = url
    for i in range(1, num+1):
        if i != 0:
            lastpage_url = get_lastpage(bb)
            df = get_articles(lastpage_url, today)
            article = article.append(df, axis)
    return article

In [4]:
def get_lastpage(url):
    
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    divs = soup.find_all("a", "btn wide")
    lastpage_url = urllib.parse.urljoin("https://www.ptt.cc", divs[1]["href"])
    return lastpage_url

In [5]:
def checkformat(soup, class_tag, data, index, link):
    
    try:
        content = soup.select(class_tag)[index].text
    except Exception as e:
        print('格式跑掉爬不到QQ')
        # print 'checkformat:',str(e)
        content = "no " + data
    return content

In [6]:
def get_content(url):
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    # content  文章內文
    try:
        content = soup.find(id="main-content").text
        target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        content = content.split(target_content)
        content = content[0].split(checkformat(soup, '.article-meta-value', 'date', 3, url))
        main_content = content[1].replace('\n', '  ')
        # print 'content:',main_content
    except Exception as e:
        main_content = 'main_content error'
        print('格式跑掉爬不到QQ')
    return main_content


In [7]:
def get_articles(url):
    
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    divs = soup.find_all("div", "r-ent")
    articles = []

    for i in range(0, len(divs)):
                   
        push = 0                
        if divs[i].find("div", "nrec").string:                    
            try:                    
                push = int(divs[i].find("div", "nrec").string)
            except ValueError:
                pass
                
   
        if divs[i]("a"):
            date = divs[i].find("div", "date").string.strip()
            href = urllib.parse.urljoin('https://www.ptt.cc/',divs[i].find('a')['href'])
            title = divs[i].find('a').string
            content  = get_content(href)
                     
                        
            articles.append({
                'date': date,
                'push': push,
                'title': title,
                'link': href,
                'content' : content
             })
    
        
    return pd.DataFrame(articles)[['date', 'push', 'title', 'link', 'content']]

In [8]:
def Crawler_PTT_Content(Board_Name , page, date = None):
    
    url  = get_board_url(Board_Name)
    article = get_articles(url)
    for i in range(1, page+1):
        print('現在正在往前爬第', i, '頁')
        if i != 0:
            lastpage_url = get_lastpage(url)
            df = get_articles(lastpage_url)
            article = article.append(df, ignore_index=True)   
            url = lastpage_url
            
    if date !=None:
        article = article[(article.date == date)]
           
    
    return article

## 爬PTT推文

In [9]:
def Crawler_PTT_Push(url):
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    divs = soup.find_all("div", id = "main-content")
    msg = divs[0].find_all("div", "push")
    articles = []
    
    for i in range(0, len(msg)):
        
        if msg[i]("span"):
            try:
                push_tag = msg[i].find("span", "f1 hl push-tag").string.replace(' ', '')
            except AttributeError:
                push_tag = msg[i].find("span", "hl push-tag").string.replace(' ', '')
                pass
        
        time = msg[i].find("span", "push-ipdatetime").string.strip()
        message = msg[i].find("span", "f3 push-content").string
        articles.append({
            'time': time,
            'push': push_tag,
            'message': message,
        })
    return pd.DataFrame(articles)[['push', 'message', 'time']]

## 爬標題DEMO

In [10]:
demo = Crawler_PTT_Content('Stock', 10)

格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 1 頁
現在正在往前爬第 2 頁
現在正在往前爬第 3 頁
現在正在往前爬第 4 頁
現在正在往前爬第 5 頁
現在正在往前爬第 6 頁
現在正在往前爬第 7 頁
現在正在往前爬第 8 頁
現在正在往前爬第 9 頁
現在正在往前爬第 10 頁


In [11]:
demo

,date,push,title,link,content
0,6/27,7,[標的] 3050 鈺德 低調多,https://www.ptt.cc/bbs/Stock/M.1530084786.A.0E...,1. 標的：3050鈺德 2. 分類：多 3. 分析/正文： 錸德...
1,6/27,2,[標的] 3040遠見,https://www.ptt.cc/bbs/Stock/M.1530085334.A.F4...,3040遠見 除權息之後大跌兩天 從48.9跌到現在40.5原本想賺股息9塊的人加...
2,6/27,0,Re: [新聞] 台幣貶到32 央行給軟釘子,https://www.ptt.cc/bbs/Stock/M.1530085653.A.61...,"講幾個重點: 1. 美國頭部中, 建議適當避險 2. 台灣比較糟, 政府..."
3,1/25,44,[公告] 精華區導覽Q&A,https://www.ptt.cc/bbs/Stock/M.1422199105.A.84...,"關鍵字查詢：使用""/""於本文搜尋 1.請將頁面跳離關鍵字區 2.輸入""/""出..."
4,12/18,0,[公告] Stock 板規 (20151218修訂),https://www.ptt.cc/bbs/Stock/M.1450418190.A.2F...,main_content error
5,6/27,66,[閒聊] 2018/06/27 盤後閒聊,https://www.ptt.cc/bbs/Stock/M.1530079231.A.79...,太熱了 公園要住不下去了 ----- Sent from JPTT on ...
6,6/27,18,[新聞] 法人看達方 今明年獲利高成長,https://www.ptt.cc/bbs/Stock/M.1530067820.A.D1...,-------------------------------發文提醒---------...
7,6/27,14,Re: [新聞] 吸引年輕人買股 學學日本,https://www.ptt.cc/bbs/Stock/M.1530069548.A.D0...,市場機制搞得這麼爛 年輕人進來也是被坑殺 充斥著內線 不平等交易 本以為顧立...
8,6/27,71,[標的] 套在125的微星 還有救嗎,https://www.ptt.cc/bbs/Stock/M.1530072497.A.C4...,1. 標的：2377 微星 2. 分類：請益 3. 分析/正文： ...
9,6/27,4,Re: [投顧] 經濟日報 比賽總覽,https://www.ptt.cc/bbs/Stock/M.1530073345.A.B6...,※ 引述《ckshchen (尖酸苛博文)》之銘言： : 郭哲榮 鈺邦/蜜望實/立隆...


## 爬推文DEMO

In [12]:
demo.link[0]

'https://www.ptt.cc/bbs/Stock/M.1530084786.A.0E6.html'

In [13]:
Crawler_PTT_Push(demo.link[0])

,push,message,time
0,推,: 0.0,06/27 15:33
1,噓,: 低調那你po什麼文,06/27 15:34
2,噓,: 低調,06/27 15:34
3,推,: 還有個二兒子錸寶,06/27 15:34
4,→,: 錸寶還沒上市,06/27 15:34
5,推,: 數來寶,06/27 15:34
6,推,: 媽寶,06/27 15:35
7,推,: 媽寶？,06/27 15:37
8,噓,: 低調幹嘛PO文,06/27 15:38
9,推,: 幫高調,06/27 15:38


## 情緒分析

In [14]:
from snownlp import SnowNLP

In [210]:
def emotion_analysis(sentence):
    df = SnowNLP(sentence)
    prob = df.sentiments
    return(prob)

In [211]:
sentence = demo["title"][0]

In [212]:
sentence

'[標的] 3050 鈺德 低調多 '

In [213]:
prob = emotion_analysis(sentence)

In [214]:
prob

0.05522534652580102

## 標題與內文評分

In [99]:
demo_group = demo.groupby("date")

In [138]:
dfff =  demo_group.get_group("6/27").reset_index(drop=True)

In [172]:
dfff.head()

,date,push,title,link,content
0,6/27,7,[標的] 3050 鈺德 低調多,https://www.ptt.cc/bbs/Stock/M.1530084786.A.0E...,1. 標的：3050鈺德 2. 分類：多 3. 分析/正文： 錸德...
1,6/27,2,[標的] 3040遠見,https://www.ptt.cc/bbs/Stock/M.1530085334.A.F4...,3040遠見 除權息之後大跌兩天 從48.9跌到現在40.5原本想賺股息9塊的人加...
2,6/27,0,Re: [新聞] 台幣貶到32 央行給軟釘子,https://www.ptt.cc/bbs/Stock/M.1530085653.A.61...,"講幾個重點: 1. 美國頭部中, 建議適當避險 2. 台灣比較糟, 政府..."
3,6/27,66,[閒聊] 2018/06/27 盤後閒聊,https://www.ptt.cc/bbs/Stock/M.1530079231.A.79...,太熱了 公園要住不下去了 ----- Sent from JPTT on ...
4,6/27,18,[新聞] 法人看達方 今明年獲利高成長,https://www.ptt.cc/bbs/Stock/M.1530067820.A.D1...,-------------------------------發文提醒---------...


In [277]:
def content_score(df_content):
    ## 分析當天有po的文
    ## 將內文轉成一句摘要，分別分析該句摘要以及標題
    ## 給予情緒分析，機率大於0.5給1，小於0.5給0
    
    score_avg = float()
    score_total = float()
    num = 0
    for i in range(0, len(df_content["content"])):
        score = float()
        ##標題
        try:
            if(emotion_analysis(df_content["title"][i]))>0.5:
                score = score + 1
                num = num + 1
            else:
                score = score - 1
                num = num + 1
        except:
            continue
        ##內文的摘要
        try:
            df = SnowNLP(df_content["content"][i])
            df_summary = df.summary(1)
            if(emotion_analysis(df_summary[0]))>0.5:
                score = score + 1
            else:
                score = score - 1
        except:
            continue
        score_total = score_total + score/2
    score_avg = score_total/num
    return score_avg, score_total, num

In [278]:
ContentScore = content_score(dfff)

In [279]:
ContentScore

(-0.45652173913043476, -21.0, 46)

## 留言評分

In [246]:
## 統整當天文的留言
msg_total = pd.DataFrame(columns=['push', 'message', 'time'])
for i in range(0, len(dfff["link"])):
    msg = Crawler_PTT_Push(dfff["link"][i])
    msg_total = pd.concat([msg_total, msg], axis=0)

In [249]:
msg_total = msg_total.reset_index(drop=True)

In [273]:
msg_total.head()

,push,message,time
0,推,: 0.0,06/27 15:33
1,噓,: 低調那你po什麼文,06/27 15:34
2,噓,: 低調,06/27 15:34
3,推,: 還有個二兒子錸寶,06/27 15:34
4,→,: 錸寶還沒上市,06/27 15:34


In [263]:
def message_score(df_message):
    score_total = float()
    score_avg = float()
    num = 0
    for i in range(0, len(df_message["message"])):
        ##留言
        try:
            if(emotion_analysis(df_message["message"][i]))>0.5:
                score_total = score_total + 1
                num = num + 1
            else:
                score_total = score_total - 1
                num = num + 1
        except:
            continue
    score_avg = score_total/num
    return score_avg, score_total, num

In [264]:
MessageScore = message_score(msg_total)

In [265]:
MessageScore

(-0.36845008273579705, -1336.0, 3626)

## 當天總分數

In [ ]:
## 假設標題與內文占六成，留言占四成

In [281]:
ContentScore[0]

-0.45652173913043476

In [282]:
MessageScore[0]

-0.36845008273579705

In [283]:
total_score = 0.6*ContentScore[0] + 0.4*MessageScore[0]

In [284]:
total_score

-0.42129307657257964